In [ ]:
from datascience import *
import numpy as np
import matplotlib

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

import warnings
warnings.simplefilter("ignore")

## Review

### The $k$-Nearest Neighbors algorithm

#### Find the “distance” between the example and one training point

##### We use $\text{D}$, the **Euclidean distance:**

$$ \text{D} = \sqrt{(x_0-x_1)^2 + (y_0-y_1)^2} $$

In [ ]:
def distance(point1, point2):
    """Returns the distance between point1 and point2
    where each argument is an array 
    consisting of the coordinates of the point"""
    return np.sum((point1-point2)**2) ** 0.5

#### Find the “distance” between the example and each observation in the training set

In [ ]:
def all_distances(training, new_row):
    """Returns an array of distances
    between each point in the training set
    and the new point (which is a row of attributes)"""
    attributes = training.drop('Class')
    def distance_from_new(row):
        return distance(make_array(new_row), make_array(row))
    return attributes.apply(distance_from_new)

#### Augment the training data table with a column containing all the distances

In [ ]:
def table_with_distances(training, new_point):
    """Augments the training table 
    with a column of distances from new_point"""
    return training.with_column('Distance', all_distances(training, new_point))

#### The smaller $\text{D}$ is, the nearer the neighbor! Find the $k$-nearest neighbors using table manipulations.

In [ ]:
def nearest(training, new_point, k):
    """Returns a table of the k rows of the augmented table
    corresponding to the k smallest distances"""
    with_dists = table_with_distances(training, new_point)
    sorted_by_distance = with_dists.sort('Distance')
    nearest_neighbors_table = sorted_by_distance.take(np.arange(k))
    return nearest_neighbors_table

#### Take a majority vote of the $k$-nearest neighbors. Assign the new point to the majority class

In [ ]:
def majority(nearest_neighbors_table, class_name):
    return nearest_neighbors_table.group(class_name).sort('count', 
                                                         descending=True).column(class_name).item(0)

#### A function to run the complete algorithm!

In [ ]:
def one_knn(training, class_name, new_point, k):
    nearest_neighbors_table = nearest(training, new_point, k)
    return majority(nearest_neighbors_table, class_name)

____

### `ckd` dataset

In [ ]:
ckd = Table.read_table('ckd.csv').relabeled('Blood Glucose Random', 'Glucose')
ckd.sample(3).show(3)

In [ ]:
ckd = ckd.select('Hemoglobin','Glucose','White Blood Cell Count', 'Class')

_____

### `banknotes` dataset

In [ ]:
banknotes = Table().read_table('banknote.csv')
banknotes.sample(3).show()

In [ ]:
first_banknote = banknotes.drop('Class').row(0)
first_banknote

**Distance in 4 dimensions**

$$ \text{D} = \sqrt{(w_0-w_1)^2 + (x_0-x_1)^2 + (y_0-y_1)^2 + (z_0-z_1)^2} $$

In [ ]:
banknotes.show(1)

In [ ]:
nearest(banknotes,first_banknote,1)

In [ ]:
patients = Table.read_table('breast-cancer.csv').drop('ID')

In [ ]:
patients.sample(5)

In [ ]:
patients.num_columns

### Exploration

In [ ]:
patients.group('Class')

In [ ]:
patients.scatter('Bland Chromatin', 'Single Epithelial Cell Size', group='Class')
plt.title('You are witnessing something called overplotting!');

Do not worry about how to use the code below. Just understand that it makes a new table containing data which enhance the previous visualization!

In [ ]:
def randomize_column(a):
    return a + np.random.normal(0.0, 0.09, size=len(a))

jittered = Table().with_columns([
        'Bland Chromatin (jittered)', 
        randomize_column(patients.column('Bland Chromatin')),
        'Single Epithelial Cell Size (jittered)', 
        randomize_column(patients.column('Single Epithelial Cell Size')),
        'Class',
        patients.column('Class')
    ])

In [ ]:
jittered.scatter(0, 1, group='Class')

**STOP**

### Prediction

#### **Task**: Split the `patients` data into `train` and `test` sets of (roughly) equal size.

##### Method 1: from yesterday - use the `sample` + `take` table methods

In [ ]:
halfway = round(patients.num_rows/2)
halfway

In [ ]:
shuffled_patients = patients.sample(with_replacement=False)

In [ ]:
train = shuffled_patients.take(np.arange(halfway))
test = shuffled_patients.take(np.arange(halfway, patients.num_rows))

##### Method 2: use the `split` table method!
- If you're confused about what's going on here - don't feel bad. This code involves the use of a data type in Python called a `tuple`, which is beyond the scope of the course. You can just use code like this to speed up the train and test split, if you wish.

In [ ]:
train, test = patients.split(k = round(patients.num_rows/2))

#### **Task**: Calculate the Euclidean distance between an example test point and every point in the training set.

In [ ]:
example = test.drop('Class').row(2)
example

**Discussion Question**

In [ ]:
table_with_distances(train, example)

#### **Task**: Find the 5-nearest neighbors!

In [ ]:
nearest(train, example, 5)

#### **Task**: Find the majority class!

In [ ]:
majority(nearest(train, example, 5), 'Class')

#### **Task**: Repeat this process for every point in the testing set!

In [ ]:
one_knn(train, 'Class', test.drop('Class').row(0),  5)

There are a few ways to do this. Today, we will use a for-loop method.

In [ ]:
def knn(training, class_name, testing, k):
    """Returns the test predictions in the test set in a table, along with their actual classifications"""
    
    predictions = make_array()
    for i in np.arange(testing.num_rows):
        predictions = np.append(predictions, one_knn(training, 
                                                     class_name, 
                                                     testing.drop(class_name).row(i), 
                                                     k))

    knn_table = Table().with_columns('Actual', testing.column(class_name),
                                     'Predicted', predictions)
    
    return knn_table 

In [ ]:
results = knn(train, 'Class', test, 5)
results

**STOP**

## New material

#### **Task**: Evaluate the accuracy of the classifier using $k=5$.

$$\text{Misclassification Rate (MCR)} = \text{proportion of testing points that are misclassified}$$

- Practice [5 min]: Find the mis-classification rate for the 5-nearest neighbors algorithm!
  - Can you find the number of misclassified points?
  - Can you find the total number of points in the `test` table?
  - If you can do both of these, you should be able to answer this question!
  - Or, see if you can use any useful Numpy methods to do the calculation more quickly!

In [ ]:
results.column('Actual') != results.column('Predicted')

#### **Task**: See where our classifier went wrong.

**Discussion Question**: How many rows and columns will the table produced by the following command have?

```
results.group(['Actual', 'Predicted'])
```


In [ ]:
results.group(['Actual', 'Predicted'])

**Discussion Question**

**STOP**

### If you are purely interested in prediction... standardize!

In [ ]:
def standard_units(x):
    return (x - np.average(x)) / np.std(x)

In [ ]:
ckd.sample(3).show(3)

In [ ]:
ckd_train,ckd_test = ckd.split(k = round(ckd.num_rows/2))

In [ ]:
ckd_results = knn(ckd_train, 'Class', ckd_test, 5)
np.average(ckd_results.column('Actual') != ckd_results.column('Predicted'))

Notice that the standardization comes after the train/test split.

In [ ]:
std_ckd_train = ckd_train.select('Class').with_columns(
    'Glucose_su', standard_units(ckd_train.column('Glucose')),
    'Hemoglobin_su', standard_units(ckd_train.column('Hemoglobin')),
    'WBC_su', standard_units(ckd_train.column('White Blood Cell Count'))
)

In [ ]:
std_ckd_test = ckd_test.select('Class').with_columns(
    'Glucose_su', standard_units(ckd_test.column('Glucose')),
    'Hemoglobin_su', standard_units(ckd_test.column('Hemoglobin')),
    'WBC_su', standard_units(ckd_test.column('White Blood Cell Count'))
)

In [ ]:
std_ckd_results = knn(std_ckd_train, 'Class', std_ckd_test, 5)
np.average(std_ckd_results.column('Actual') != std_ckd_results.column('Predicted'))

## Appendix

### Which $k$ should we choose?

We can figure out which one based on the evaluation metric, $\text{MCR}$. 

This step should be performed on the training set, *not* the testing set. Usually, we end up splitting the original training set into two parts: 

- "new" training set
- validation set.

In [ ]:
new_train, validation = train.split(k = round(train.num_rows*(7/10)))

In [ ]:
new_train.num_rows

In [ ]:
validation.num_rows

In [ ]:
def which_k(training, class_name, validation, k_values):

    mc_rates = make_array()

    for i in np.arange(np.size(k_values)):

        results = knn(training, class_name, validation, k_values.item(i))
        new_rate = np.average(results.column('Actual') != results.column('Predicted'))
        mc_rates = np.append(mc_rates, new_rate)

    k_table = Table().with_columns("k", k_values,
                                   "MCR", mc_rates)
                             
    return k_table

In [ ]:
k_s = make_array(1,3,5,7,9,11,13,15,17,19,21,23,25)

In [ ]:
k_table = which_k(new_train, 'Class', validation, k_s)

In [ ]:
k_table

**Discussion Question**: What plot type would work best to visualize how the $\text{MCR}$ increases and decreases as the number of neighbors increases?

In [ ]:
k_table.plot('k','MCR')
plt.title('The best k for the job is...?');

#### **Task**: Evaluate our classifier's performance on the test set using the best $k$: ($k = 3$).

In [ ]:
results = knn(train, 'Class', test, 9)

In [ ]:
np.average(results.column('Actual') != results.column('Predicted'))